In [1]:
# Step 0. Import libraries and custom modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Step 1. Load data, show info and sample
# 1.1 Read data
df_raw = pd.read_excel(
   'https://github.com/datagy/mediumdata/raw/master/sample_pivot.xlsx', 
   parse_dates=['Date']
)
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    1000 non-null   datetime64[ns]
 1   Region  1000 non-null   object        
 2   Type    1000 non-null   object        
 3   Units   911 non-null    float64       
 4   Sales   1000 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 39.2+ KB


In [3]:
# 1.2 Show sample
df_raw.sample(5, random_state=1234)

,Date,Region,Type,Units,Sales
681,2020-11-27,East,Children's Clothing,32.0,342
990,2020-12-17,North,Children's Clothing,7.0,217
155,2020-06-13,East,Children's Clothing,30.0,336
768,2020-06-25,North,Women's Clothing,5.0,644
438,2020-10-07,North,Women's Clothing,30.0,462


In [4]:
# Step 2. Transform to standard format
# 2.1 Create a pandas chain to convert to final form
df = (
    df_raw
    .copy()
    .set_axis(
        df_raw
        .columns.str.lower()
        .str.replace(' ','_')
        .str.replace(r'\W','',regex=True)
        .str.slice(0,40), axis=1
    )
    .astype({'region':'category', 
             'type':'category'})
    .assign(
        date = lambda x: pd.to_datetime(x['date'], format='%Y-%m-%d')
    )
    .set_index('date')
)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2020-07-11 to 2020-08-17
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   region  1000 non-null   category
 1   type    1000 non-null   category
 2   units   911 non-null    float64 
 3   sales   1000 non-null   int64   
dtypes: category(2), float64(1), int64(1)
memory usage: 25.7 KB


In [17]:
display(df)

,region,type,units,sales
date,,,,
2020-07-11,East,Children's Clothing,18.0,306
2020-09-23,North,Children's Clothing,14.0,448
2020-04-02,South,Women's Clothing,17.0,425
2020-02-28,East,Children's Clothing,26.0,832
2020-03-19,West,Women's Clothing,3.0,33
...,...,...,...,...
2020-02-11,East,Children's Clothing,35.0,735
2020-12-25,North,Men's Clothing,NaN,1155
2020-08-31,South,Men's Clothing,13.0,208


In [20]:
# Step 3. Perform busines analysis
# 3.1 Get the summary, for categorical and numerical columns
display(df.describe(include='number').T)
display(df.describe(include='category').T)

,count,mean,std,min,25%,50%,75%,max
units,911.0,19.638858,9.471309,3.0,12.0,20.0,28.0,35.0
sales,1000.0,427.254000,253.441362,33.0,224.0,380.0,575.0,1155.0


,count,unique,top,freq
region,1000,4,East,411
type,1000,3,Women's Clothing,424


In [21]:
# 3.2 Get the sum of total sales
df['sales'].sum()

np.int64(427254)

In [22]:
# 3.3 Get the average of sales
df['sales'].mean()

np.float64(427.254)

In [25]:
# 3.4 Get the sum of sales by region
df.groupby('region', observed=False)['sales'].sum()

region
East     167763
North    138700
South     59315
West      61476
Name: sales, dtype: int64

In [26]:
# 3.4 Get the average sales by region
df.groupby('region', observed=False)['sales'].mean()

region
East     408.182482
North    438.924051
South    432.956204
West     452.029412
Name: sales, dtype: float64

In [27]:
# 3.5 Get the sum, mean and count of sales by region
df.groupby('region', observed=False)['sales'].agg(['sum','mean','count'])

,sum,mean,count
region,,,
East,167763,408.182482,411
North,138700,438.924051,316
South,59315,432.956204,137
West,61476,452.029412,136


In [33]:
df.groupby('region', observed=False)['sales'].agg(['sum','mean','count']).reset_index()

,region,sum,mean,count
0,East,167763,408.182482,411
1,North,138700,438.924051,316
2,South,59315,432.956204,137
3,West,61476,452.029412,136


In [36]:
# 3.6 Create a pivot table that gets total units by type
df.pivot_table(index='type', values='units', aggfunc='sum', observed=False, margins=True)

,units
type,
Children's Clothing,5887.0
Men's Clothing,3974.0
Women's Clothing,8030.0
All,17891.0


In [38]:
# 3.7 Display the dataset showing only the East region
display(df.loc[df['region']=='East'])

,region,type,units,sales
date,,,,
2020-07-11,East,Children's Clothing,18.0,306
2020-02-28,East,Children's Clothing,26.0,832
2020-03-25,East,Women's Clothing,29.0,609
2020-11-03,East,Children's Clothing,34.0,374
2020-05-01,East,Men's Clothing,10.0,140
...,...,...,...,...
2020-03-20,East,Men's Clothing,14.0,238
2020-06-05,East,Children's Clothing,11.0,220
2020-05-04,East,Men's Clothing,6.0,108


In [41]:
# 3.8 Display the dataset showing only the East and West regions
display(df[(df['region']=='East')|(df['region']=='West')])

,region,type,units,sales
date,,,,
2020-07-11,East,Children's Clothing,18.0,306
2020-02-28,East,Children's Clothing,26.0,832
2020-03-19,West,Women's Clothing,3.0,33
2020-03-25,East,Women's Clothing,29.0,609
2020-11-03,East,Children's Clothing,34.0,374
...,...,...,...,...
2020-02-08,West,Men's Clothing,32.0,928
2020-05-04,East,Men's Clothing,6.0,108
2020-11-17,West,Men's Clothing,27.0,486


In [42]:
# 3.9 Display the dataset showing East sales with units over 30 
display(df[(df['region']=='East')&(df['units']>30)])

,region,type,units,sales
date,,,,
2020-11-03,East,Children's Clothing,34.0,374
2020-06-12,East,Women's Clothing,35.0,1050
2020-05-11,East,Men's Clothing,35.0,700
2020-06-14,East,Men's Clothing,32.0,416
2020-10-28,East,Men's Clothing,34.0,986
...,...,...,...,...
2020-02-24,East,Women's Clothing,32.0,693
2020-08-13,East,Men's Clothing,32.0,132
2020-10-04,East,Men's Clothing,35.0,350


In [43]:
# 3.10 A data corruption has been detected. Add 1 to all units. 
df_new = df.copy()
df_new['units'] = df_new['units'] + 1


In [ ]:
# 3.11 Another issue was detected, in West region add 2 to all units
df_2new = df.copy()
df_2new
# df_2new['units'] = df_2new.apply(lambda row: row['units']+2 if row['region']=='West' else row['units'], axis=1)

In [48]:
# 3.11 Another issue was detected, in West region add 2 to all units
df_3new = df.copy()
df_3new.loc[df_3new['region'] == 'West', 'units'] += 2